In [1]:
import gradio as gr
import pandas as pd
from transformers import TapasTokenizer, TapasForQuestionAnswering
from transformers import AutoTokenizer, AutoModelForTableQuestionAnswering
from transformers import pipeline

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_answer(dropdown, question):
    df = pd.DataFrame()
    if dropdown == "Master Information Systems":
        df = pd.read_excel("C:/Users/michi/enterpriseAI_michi/enterprise-ai-project/module_guide_tableQA/felix_playground_SQA_Training/MS_IS_all_modules_orginal_15_rows_cleaned.xlsx")
    elif dropdown == "Bachelor Information Systems":
        df = pd.read_excel("BA_IS_all_modules.xslx")
    elif dropdown == "Master Management":
        df = pd.read_excel("MA_Management_all_modules.xslx")
    df = df.astype(str)
    print(question)
    question = str(question)
    print(df.shape)
    question = [question]
    tqa = pipeline(task="table-question-answering", 
                model="google/tapas-base-finetuned-wtq")
    
    results = tqa(table=df, query=question)
    print(results)
    cells_input = results['cells']
    cells_input = str(cells_input)
    cells_input = cells_input.replace("[", "")
    cells_input = cells_input.replace("]", "")
    cells_input = cells_input.replace("'", "")

    print(cells_input)
    print(results)
    html_string_short = f"<h2>Short Answer:</h2><p>{cells_input}</p>"
    row_numbers = [coord[0] for coord in results['coordinates']]
    df_short = df.iloc[row_numbers]
    df_short = df_short.dropna(axis=1, how='all')
    df_short = df_short.loc[:, (df_short != '--').any(axis=0)]
    html_table = f"<h2>Complete Module(s):</h2><p>{df_short.to_html(index=False)}</p>"
    return html_string_short, html_table

with gr.Blocks() as demo:
    
    gr.HTML("<h1>Your Module Guide Assistant</h1><p>Select a module guide from the dropdown and enter your question below.</p>")
    table = gr.Dropdown(["Master Information Systems", "Bachelor Information Systems", "Master Management"], label="Module Guide", value="Master Information Systems")
    question = gr.Textbox(label="Question", value="How many ECTS credits does the project seminar have?")
    ask_btn = gr.Button("Ask The Assistant")
    gr.HTML("<hr>")

    inputs = [table, question]
    output_question = gr.HTML(label="Answer")
    outout_full_module = gr.HTML(label="Detailed Description")
    outputs = [output_question, outout_full_module]
 
    ask_btn.click(fn=get_answer, inputs=inputs, outputs=outputs, api_name="greet")

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


How many ECTS credits does the project seminar have?
(15, 17)
{'answer': 'COUNT > 15', 'coordinates': [(2, 4)], 'cells': ['15'], 'aggregator': 'COUNT'}
15
{'answer': 'COUNT > 15', 'coordinates': [(2, 4)], 'cells': ['15'], 'aggregator': 'COUNT'}
